In [100]:
import warnings
warnings.filterwarnings('ignore')

In [101]:
task_set=[]
with open('task_setEDF.txt', 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('['):
            line = line[1:]
        if line.endswith(']'):
            line = line[:-1]
        values = line.split(',')
        task_set.append([values[1][2:len(values[1])-1],float(values[2]),float(values[3]),int(values[4]),int(values[5])])
print(task_set)

[['T1', 0.0, 3.0, 20, 20], ['T2', 2.0, 2.0, 5, 5], ['T3', 1.0, 2.0, 10, 10]]


In [102]:
def schedulability(task_set):
    util=0;
    for task in task_set:
        util+=(task[2]/min(task[3],task[4]))
    return (util<=1)

In [103]:
import math
def hyperperiod(periods):
    lcm_ = periods[0]
    for i in range(1, len(periods)):
        lcm_ = math.lcm(lcm_, periods[i])
    return lcm_

In [104]:
import pandas as pd
df=pd.DataFrame(task_set,columns=["Task","release_time","ex_time","period","Deadline"])
df

,Task,release_time,ex_time,period,Deadline
0,T1,0.0,3.0,20,20
1,T2,2.0,2.0,5,5
2,T3,1.0,2.0,10,10


In [105]:
df['time_left'] = df['ex_time']
df['rel_deadline'] = df['Deadline']
df

,Task,release_time,ex_time,period,Deadline,time_left,rel_deadline
0,T1,0.0,3.0,20,20,3.0,20
1,T2,2.0,2.0,5,5,2.0,5
2,T3,1.0,2.0,10,10,2.0,10


In [106]:
DMS=[]

In [107]:
if schedulability(task_set):
    for i in range(hyperperiod(df.period)):
        rem=df[df["release_time"]<=i]
        rem=rem[rem["time_left"]>0]
        if len(rem)>0:
            s_task=rem.sort_values('rel_deadline').index[0]
            df.loc[s_task,"time_left"]=df.loc[s_task]["time_left"]-1.0
            if i>0 and DMS[-1]['Task']==df.loc[s_task]["Task"] and DMS[-1]['end']==i :
                DMS[-1].update({'end':i+1,'length':DMS[-1]['length']+1})
            else : 
                DMS += [{'Task': df.loc[s_task]["Task"],'start': i,'end': i+1,'length': 1}]
        arrived=df[(i+1)%df['period']==0].index
        df.loc[arrived, 'time_left'] = df.loc[arrived, 'ex_time']
        df.loc[arrived, 'rel_deadline'] = df.loc[arrived, 'Deadline']+df.loc[arrived, 'period']
else:
    print("Not Schedulabe using DMA!! :/")

In [108]:
DMS

[{'Task': 'T1', 'start': 0, 'end': 1, 'length': 1},
 {'Task': 'T3', 'start': 1, 'end': 2, 'length': 1},
 {'Task': 'T2', 'start': 2, 'end': 4, 'length': 2},
 {'Task': 'T3', 'start': 4, 'end': 5, 'length': 1},
 {'Task': 'T2', 'start': 5, 'end': 7, 'length': 2},
 {'Task': 'T1', 'start': 7, 'end': 9, 'length': 2},
 {'Task': 'T2', 'start': 10, 'end': 12, 'length': 2},
 {'Task': 'T3', 'start': 12, 'end': 14, 'length': 2},
 {'Task': 'T2', 'start': 15, 'end': 17, 'length': 2}]

In [109]:
DMS=pd.DataFrame(DMS)

In [110]:
if schedulability(task_set): print("Yaay!!  Schedulabe using DMA :")
print("------------------------------------------------------------------------------")
DMS

Yaay!!  Schedulabe using DMA :
------------------------------------------------------------------------------


,Task,start,end,length
0,T1,0,1,1
1,T3,1,2,1
2,T2,2,4,2
3,T3,4,5,1
4,T2,5,7,2
5,T1,7,9,2
6,T2,10,12,2
7,T3,12,14,2
8,T2,15,17,2
